In [2]:
%cd /home/jnainani_umass_edu/codellm/MechInterpCodeLLMs

/home/jnainani_umass_edu/codellm/MechInterpCodeLLMs


In [3]:
import transformer_lens
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
import os
import time
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional, Callable
from typing_extensions import Literal
from functools import partial
import copy
import itertools
import json

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets

import argparse

# THIS IS A LOCAL (MODIFIED) VERSION OF TRANSFORMER_LENS - UNINSTALL PIP/CONDA VERSION BEFORE USE!
import transformer_lens
import transformer_lens.utils as utils
import transformer_lens.patching as patching
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)

import utils_patch.patch_helper_functions as helper

import os
from config import HF_TOKEN, HF_PATH

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["TRANSFORMERS_CACHE"] = HF_PATH
os.environ["HF_DATASETS_CACHE"] = HF_PATH
os.environ["HF_HOME"] = HF_PATH

num_gpus = torch.cuda.device_count()
device_id = 0
if num_gpus > 0:
    device = "cuda:0"
else:
    device = "cpu"

    
def check_gpu_memory(max_alloc=0.9):
    if not torch.cuda.is_available():
        return
    global device_id, device
    print("Primary device:", device)
    torch.cuda.empty_cache()
    max_alloc = 1 if max_alloc > 1 else max_alloc
    for gpu in range(num_gpus):
        memory_reserved = torch.cuda.memory_reserved(device=gpu)
        memory_allocated = torch.cuda.memory_allocated(device=gpu)
        total_memory = torch.cuda.get_device_properties(gpu).total_memory 
        print(f"GPU {gpu}: {total_memory / (1024**2):.2f} MB  Allocated: {memory_allocated / (1024**2):.2f} MB  Reserved: {memory_reserved / (1024**2):.2f} MB")
                
        # Check if the current GPU is getting too full, and if so we switch the primary device to the next GPU
        if memory_reserved > max_alloc * total_memory:
            if device_id < num_gpus - 1:
                device_id += 1
                device = f"cuda:{device_id}"
                print(f"Switching primary device to {device}")
            else:
                print("Cannot switch primary device, all GPUs are nearly full")

print("Number of GPUs:", num_gpus)
check_gpu_memory()

Number of GPUs: 1
Primary device: cuda:0
GPU 0: 81050.62 MB  Allocated: 0.00 MB  Reserved: 0.00 MB


In [5]:
model = HookedTransformer.from_pretrained("CodeLlama-7b-hf", n_devices=num_gpus)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model CodeLlama-7b-hf into HookedTransformer


In [83]:
prompts = ["""X = "-"\nY = "+"\nZ = "*"\nW = "\"\nthe name of the variable with value "*" is """, 
           """X = "-"\nY = "+"\nZ = "*"\nW = "\"\nthe name of the variable with value "+" is """, 
           """X = "-"\nY = "+"\nZ = "*"\nW = "\"\nthe name of the variable with value "-" is """]
answers = ["cat", "bird", "dog", "horse"]

In [79]:
model.generate(prompts[0], max_new_tokens=5)

  0%|          | 0/5 [00:00<?, ?it/s]

'lyh = "-"\njnh = "+"\nthj = "*"\nbgd = ""\nthe name of the variable with value "*" is 4\n\\end{'

In [84]:
for prompt in prompts: 
    clean_tokens = model.to_tokens(prompt)
    logits = model(clean_tokens)[:, -1, :]
    model_answers = torch.argmax(logits, dim=-1)
    print(model_answers)
    print(model.to_string(model_answers))

tensor([796], device='cuda:0')
Z
tensor([612], device='cuda:0')
Y
tensor([29896], device='cuda:0')
1


In [55]:
def evaluate_next_token_accuracy(prompts, answers, model):
    # total_examples = len(prompts)
    correct_predictions = 0
    with torch.no_grad():
        clean_tokens = model.to_tokens(prompts)
        logits = model(clean_tokens)[:, -1, :]
        model_answers = torch.argmax(logits, dim=-1)
    for ind in range(len(model_answers)):
        if model.to_string(model_answers[ind]) == answers[ind]:
            correct_predictions+=1
    return correct_predictions / len(model_answers)

evaluate_next_token_accuracy(prompts, answers, model)
    

0.5

In [36]:
logits = model(prompts)[:, -1, :]
predicted_token_id = torch.argmax(logits, dim=-1).item()
model.to_string(predicted_token_id)

RuntimeError: a Tensor with 2 elements cannot be converted to Scalar

In [23]:
model.to_string(logits)

ValueError: Invalid shape passed in: torch.Size([1, 32, 32016])

In [99]:
with open('data/prompt_dataset_zero_shot.json', 'r') as f:
    data = json.load(f)

# Take the first few dictionaries (e.g., first 3)
subset = data[9:]

# Initialize lists
prompts = []
answers = []

# Extract prompts and outputs
outputs = []
for ind, item in enumerate(subset):
    # if ind in [0, 1, 3, 4, 5, 6, 7, 9]:
    prompts.append(item["prompt"])
    outputs.append(item["output"])

In [108]:
prompts = ["""X = "-"\nY = "+"\nZ = "*"\nW = "<"\nthe name of the variable with value "*" is """, 
           """X = "-"\nY = "+"\nZ = "*"\nW = "<"\nthe name of the variable with value "+" is """, 
           """X = "-"\nY = "+"\nZ = "*"\nW = "<"\nthe name of the variable with value "-" is """]
answers = ["Z", "Y", "X"]
print(prompts[0])

X = "-"
Y = "+"
Z = "*"
W = "<"
the name of the variable with value "*" is 


In [113]:
def evaluate_next_token_accuracy(prompts, answers, model):
    # total_examples = len(prompts)
    correct_predictions = 0
    for ind in range(len(prompts)):
        # print(prompts[ind])
        with torch.no_grad():
            clean_tokens = model.to_tokens(prompts[ind])
            logits = model(clean_tokens)[:, -1, :]
            model_answers = torch.argmax(logits, dim=-1)
            print("pred: ",model.to_string(model_answers))
            if model.to_string(model_answers) == answers[ind]:
                # print("pred: ",model.to_string(model_answers))
                print("ans: ",answers[ind])
                correct_predictions+=1
    return correct_predictions / len(prompts)

# evaluate_next_token_accuracy(prompts, answers, model)

In [114]:
evaluate_next_token_accuracy(prompts, answers, model)

pred:  Z
ans:  Z
pred:  Y
ans:  Y
pred:  1


0.6666666666666666

In [ ]:

import random
import json

def generate_prompts(num_prompts=400):
    operators = ['+', '-', '*', '**', '%', '<', '>', '<=', '>=', '==']
    functions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
    prompts = []
    
    for _ in range(num_prompts):
        selected_operators = random.sample(operators, 11)
        selected_functions = random.sample(functions, 11)
        
        prompt = "For this set of functions, return the function which uses the specified operator.\n\n"
        
        examples = []
        op_to_func_mapping = {}

        for op, func in zip(selected_operators, selected_functions):
            prompt += f"def {func}(a, b):\n"
            prompt += f"    return a {op} b\n"
            examples.append([func,op])
            op_to_func_mapping[op] = func
        

        
        prompt += f"\nExample: the name of the function that uses the '{examples[0][1]}' operator is "
        prompt += examples[0][0]
        prompt += f"\nExample: the name of the function that uses the '{examples[1][1]}' operator is "
        prompt += examples[1][0]

        selected_operators.remove(examples[0][1])
        selected_operators.remove(examples[1][1])

        chosen_op = random.sample(selected_operators, 1)[0]

        prompt += f"\nExample: the name of the function that uses the '{chosen_op}' operator is "

        
        prompts.append({"prompt": prompt, "output": op_to_func_mapping[chosen_op]})
    
    print(prompts[0]['prompt'])
    print(prompts[0]['output'])
    with open("prompts_dataset.json", "w") as f:
        json.dump(prompts, f, indent=4)

generate_prompts()

In [ ]:

import random
import json

def generate_prompts(num_prompts=400):
    operators = ['+', '-', '*', '**', '%', '<', '>', '<=', '>=', '==']
    functions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
    prompts = []
    
    for _ in range(num_prompts):
        selected_operators = random.sample(operators, 11)
        selected_functions = random.sample(functions, 11)
        
        prompt = "For this set of functions, return the variable name containing the specified operator.\n\n"
        
        examples = []
        op_to_func_mapping = {}

        for op, func in zip(selected_operators, selected_functions):
            prompt += f"{func} = {op}\n"
            # prompt += f"    return a {op} b\n"
            examples.append([func,op])
            op_to_func_mapping[op] = func
        

        
        # prompt += f"\nExample: the name of the function that uses the '{examples[0][1]}' operator is "
        # prompt += examples[0][0]
        # prompt += f"\nExample: the name of the function that uses the '{examples[1][1]}' operator is "
        # prompt += examples[1][0]

        # selected_operators.remove(examples[0][1])
        # selected_operators.remove(examples[1][1])

        chosen_op = random.sample(selected_operators, 1)[0]

        prompt += f"\nThe name of the variable that contains the '{chosen_op}' operator is "

        
        prompts.append({"prompt": prompt, "output": op_to_func_mapping[chosen_op]})
    
    print(prompts[0]['prompt'])
    print(prompts[0]['output'])
    with open("prompts_dataset.json", "w") as f:
        json.dump(prompts, f, indent=4)

generate_prompts()